In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
import can
import time
import io
import cantools
from IPython.display import clear_output, display
from toolkit.common.constants import *

In [ ]:
# Define the CAN interface (You need to set this to your specific CAN interface)
CAN_INTERFACE = 'PCAN_USBBUS1'  # Replace 'can0' with your CAN interface name (e.g., 'can0', 'can1', 'vcan0', etc.)

# Initialize the CAN Bus
bus = can.interface.Bus(channel=CAN_INTERFACE, bustype='pcan', bitrate=500000)

dbc_file_path = 'sre6_motec_can1_bms.dbc'

# Load the DBC file
db = cantools.database.load_file(dbc_file_path)

In [ ]:
INITIAL_TIME = time.time()

# Create the Plotly figure
fig = make_subplots(rows=2, cols=1, subplot_titles=("CAN Data"))

# row 1 is the cell voltages and row 2 is the cell temperatures
# there are 8 modules with 12 cells per module and 4 temperature sensors per module
# each cell voltage or temperature sensor is a separate trace with time stamps

class Trace:
    def __init__(self, name, row, can_name, i):
        self.name = name
        self.ind = i
        self.row = row
        self.time_data = []
        self.data = []
        self.time_data_raw = []
        self.data_raw = []
        self.can_name = can_name

    def update(self, time, data):
        self.time_data_raw.append(time)
        self.data_raw.append(data)
        # set time and data to the last 1000 data points
        self.time_data = self.time_data_raw[-100:]
        self.data = self.data_raw[-100:]
        

traces = {}

cell_names = [("1-04", "BMS_M1_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("1-03", "BMS_M1_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("1-02", "BMS_M1_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("1-01", "BMS_M1_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"), 
               ("2-04", "BMS_M2_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("2-03", "BMS_M2_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("2-02", "BMS_M2_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("2-01", "BMS_M2_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"), 
               ("3-04", "BMS_M3_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("3-03", "BMS_M3_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("3-02", "BMS_M3_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("3-01", "BMS_M3_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"), 
               ("4-04", "BMS_M4_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("4-03", "BMS_M4_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("4-02", "BMS_M4_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("4-01", "BMS_M4_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"), 
               ("5-04", "BMS_M5_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("5-03", "BMS_M5_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("5-02", "BMS_M5_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("5-01", "BMS_M5_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"), 
               ("6-04", "BMS_M6_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("6-03", "BMS_M6_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("6-02", "BMS_M6_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("6-01", "BMS_M6_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"), 
               ("7-04", "BMS_M7_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("7-03", "BMS_M7_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("7-02", "BMS_M7_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("7-01", "BMS_M7_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"), 
               ("8-04", "BMS_M8_Cell_Voltage_Data_1_BMS_Cell_4_Voltage"), ("8-03", "BMS_M8_Cell_Voltage_Data_1_BMS_Cell_3_Voltage"), ("8-02", "BMS_M8_Cell_Voltage_Data_1_BMS_Cell_2_Voltage"), ("8-01", "BMS_M8_Cell_Voltage_Data_1_BMS_Cell_1_Voltage"),
               ("1-08", "BMS_M1_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("1-07", "BMS_M1_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("1-06", "BMS_M1_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("1-05", "BMS_M1_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("2-08", "BMS_M2_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("2-07", "BMS_M2_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("2-06", "BMS_M2_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("2-05", "BMS_M2_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("3-08", "BMS_M3_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("3-07", "BMS_M3_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("3-06", "BMS_M3_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("3-05", "BMS_M3_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("4-08", "BMS_M4_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("4-07", "BMS_M4_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("4-06", "BMS_M4_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("4-05", "BMS_M4_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("5-08", "BMS_M5_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("5-07", "BMS_M5_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("5-06", "BMS_M5_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("5-05", "BMS_M5_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("6-08", "BMS_M6_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("6-07", "BMS_M6_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("6-06", "BMS_M6_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("6-05", "BMS_M6_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("7-08", "BMS_M7_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("7-07", "BMS_M7_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("7-06", "BMS_M7_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("7-05", "BMS_M7_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("8-08", "BMS_M8_Cell_Voltage_Data_2_BMS_Cell_8_Voltage"), ("8-07", "BMS_M8_Cell_Voltage_Data_2_BMS_Cell_7_Voltage"), ("8-06", "BMS_M8_Cell_Voltage_Data_2_BMS_Cell_6_Voltage"), ("8-05", "BMS_M8_Cell_Voltage_Data_2_BMS_Cell_5_Voltage"), 
               ("1-12", "BMS_M1_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("1-11", "BMS_M1_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("1-10", "BMS_M1_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("1-09", "BMS_M1_Cell_Voltage_Data_3_BMS_Cell_9_Voltage"), 
               ("2-12", "BMS_M2_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("2-11", "BMS_M2_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("2-10", "BMS_M2_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("2-09", "BMS_M2_Cell_Voltage_Data_3_BMS_Cell_9_Voltage"), 
               ("3-12", "BMS_M3_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("3-11", "BMS_M3_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("3-10", "BMS_M3_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("3-09", "BMS_M3_Cell_Voltage_Data_3_BMS_Cell_9_Voltage"), 
               ("4-12", "BMS_M4_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("4-11", "BMS_M4_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("4-10", "BMS_M4_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("4-09", "BMS_M4_Cell_Voltage_Data_3_BMS_Cell_9_Voltage"), 
               ("5-12", "BMS_M5_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("5-11", "BMS_M5_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("5-10", "BMS_M5_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("5-09", "BMS_M5_Cell_Voltage_Data_3_BMS_Cell_9_Voltage"), 
               ("6-12", "BMS_M6_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("6-11", "BMS_M6_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("6-10", "BMS_M6_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("6-09", "BMS_M6_Cell_Voltage_Data_3_BMS_Cell_9_Voltage"), 
               ("7-12", "BMS_M7_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("7-11", "BMS_M7_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("7-10", "BMS_M7_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("7-09", "BMS_M7_Cell_Voltage_Data_3_BMS_Cell_9_Voltage"), 
               ("8-12", "BMS_M8_Cell_Voltage_Data_3_BMS_Cell_12_Voltage"), ("8-11", "BMS_M8_Cell_Voltage_Data_3_BMS_Cell_11_Voltage"), ("8-10", "BMS_M8_Cell_Voltage_Data_3_BMS_Cell_10_Voltage"), ("8-09", "BMS_M8_Cell_Voltage_Data_3_BMS_Cell_9_Voltage")]

temp_names = [("1-04T", "BMS_M1_Cell_Temp_Data_BMS_Section_4_Temp"), ("1-03T", "BMS_M1_Cell_Temp_Data_BMS_Section_3_Temp"), ("1-02T", "BMS_M1_Cell_Temp_Data_BMS_Section_2_Temp"), ("1-01T", "BMS_M1_Cell_Temp_Data_BMS_Section_1_Temp"),
              ("2-04T", "BMS_M2_Cell_Temp_Data_BMS_Section_4_Temp"), ("2-03T", "BMS_M2_Cell_Temp_Data_BMS_Section_3_Temp"), ("2-02T", "BMS_M2_Cell_Temp_Data_BMS_Section_2_Temp"), ("2-01T", "BMS_M2_Cell_Temp_Data_BMS_Section_1_Temp"),
              ("3-04T", "BMS_M3_Cell_Temp_Data_BMS_Section_4_Temp"), ("3-03T", "BMS_M3_Cell_Temp_Data_BMS_Section_3_Temp"), ("3-02T", "BMS_M3_Cell_Temp_Data_BMS_Section_2_Temp"), ("3-01T", "BMS_M3_Cell_Temp_Data_BMS_Section_1_Temp"),
              ("4-04T", "BMS_M4_Cell_Temp_Data_BMS_Section_4_Temp"), ("4-03T", "BMS_M4_Cell_Temp_Data_BMS_Section_3_Temp"), ("4-02T", "BMS_M4_Cell_Temp_Data_BMS_Section_2_Temp"), ("4-01T", "BMS_M4_Cell_Temp_Data_BMS_Section_1_Temp"),
              ("5-04T", "BMS_M5_Cell_Temp_Data_BMS_Section_4_Temp"), ("5-03T", "BMS_M5_Cell_Temp_Data_BMS_Section_3_Temp"), ("5-02T", "BMS_M5_Cell_Temp_Data_BMS_Section_2_Temp"), ("5-01T", "BMS_M5_Cell_Temp_Data_BMS_Section_1_Temp"),
              ("6-04T", "BMS_M6_Cell_Temp_Data_BMS_Section_4_Temp"), ("6-03T", "BMS_M6_Cell_Temp_Data_BMS_Section_3_Temp"), ("6-02T", "BMS_M6_Cell_Temp_Data_BMS_Section_2_Temp"), ("6-01T", "BMS_M6_Cell_Temp_Data_BMS_Section_1_Temp"),
              ("7-04T", "BMS_M7_Cell_Temp_Data_BMS_Section_4_Temp"), ("7-03T", "BMS_M7_Cell_Temp_Data_BMS_Section_3_Temp"), ("7-02T", "BMS_M7_Cell_Temp_Data_BMS_Section_2_Temp"), ("7-01T", "BMS_M7_Cell_Temp_Data_BMS_Section_1_Temp"),
              ("8-04T", "BMS_M8_Cell_Temp_Data_BMS_Section_4_Temp"), ("8-03T", "BMS_M8_Cell_Temp_Data_BMS_Section_3_Temp"), ("8-02T", "BMS_M8_Cell_Temp_Data_BMS_Section_2_Temp"), ("8-01T", "BMS_M8_Cell_Temp_Data_BMS_Section_1_Temp")]

# sort the cell names by module and cell number
cell_names.sort(key=lambda x: (int(x[0].split("-")[0]), int(x[0].split("-")[1])))

# sort the temp names by module and temp sensor number
temp_names.sort(key=lambda x: (int(x[0].split("-")[0]), int(x[0].split("-")[1][:-1])))

i = 0
for trace_name in cell_names:
    trace = Trace(trace_name[0], 1, trace_name[1], i)
    i += 1
    traces[trace_name[1]] = trace
    fig.add_trace(go.Scattergl(x=[], y=[], mode='lines', name=trace.name), row=trace.row, col=1)

for trace_name in temp_names:
    trace = Trace(trace_name[0], 2, trace_name[1], i)
    i += 1
    traces[trace_name[1]] = trace
    fig.add_trace(go.Scattergl(x=[], y=[], mode='lines', name=trace.name), row=trace.row, col=1)

fig.update_layout(height=800, width=1000, title_text="CAN Data")
fig.update_yaxes(title_text="Voltage (V)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.update_yaxes(title_text="Temperature (C)", row=2, col=1, range=[0, 50])
fig.update_xaxes(title_text="Time (s)", row=2, col=1)

print("Plot created")

# Update the plot function
def update_plot():
    # Add the new data point to the lists
    message = bus.recv()
    cur_time = time.time() - INITIAL_TIME
    updated_traces = []
    try:
        decoded_message = db.decode_message(message.arbitration_id, message.data)
        message_name = db.get_message_by_frame_id(message.arbitration_id).name
        # prefix all of the signals with the message name
        # print(message_name)
        for signal in decoded_message:
            # find if there is a trace with the same name
            tr_name = message_name + "_" + signal
            # print(tr_name)
            if tr_name in traces.keys():
                # print(f"{tr_name} is in the database")
                traces[tr_name].update(cur_time, decoded_message[signal])
                updated_traces.append(tr_name)
    except Exception as e:
        pass
        # print(e)
        # print(f"{message.arbitration_id} is not in the database")
    # print(updated_traces)
    # # Append the new data points to the existing data in each trace
    for trace_name in updated_traces:
        trace_ind = traces[trace_name].ind
        fig.data[trace_ind].x = traces[trace_name].time_data
        fig.data[trace_ind].y = traces[trace_name].data
        # print(f"{trace_name}:\t{traces[trace_name].data[-1]:.3f}")

# Start the continuous update loop
plot_timer = time.time()
while True:
    update_plot()
    if time.time() - plot_timer > 5:
        plot_timer = time.time()
        clear_output(wait=True)
        display(fig)
        # save the data to a csv file with a timestamp and data column for each trace
        # with open("charger_data.csv", "a") as f:
        #     f.write(f"{time.time() - INITIAL_TIME:.3f},")
        #     for trace_name in traces.keys():
        #         f.write(f"{traces[trace_name].data[-1]:.3f},")
        #     f.write("\n")

        
    time.sleep(0.01)  # Adjust this delay to control the update rate of the plot

In [ ]:
with open("charger_data_4.csv", "a") as f:
    f.write(f"{time.time() - INITIAL_TIME:.3f},")
    for trace_name in traces.keys():
        f.write(f"{trace_name}_time,")
        for data in traces[trace_name].time_data_raw:
            f.write(f"{data:.3f},")
        f.write(f"\n{trace_name}_data,")
        for data in traces[trace_name].data_raw:
            f.write(f"{data:.3f},")
        f.write("\n")

In [ ]:
# Start the continuous update loop
while True:
    message = bus.recv()
    # Decode the received CAN message using the DBC file
    decoded_message = db.decode_message(message.arbitration_id, message.data)

    # Access individual signal values
    if 'signal_name' in decoded_message:
        signal_value = decoded_message['signal_name']
        print(f"Received Value of 'signal_name': {signal_value}")